In [ ]:
import pandas as pd
from padelpy import from_smiles

def calculate_molecular_descriptors(descriptor_list, output_filename='all_descriptors.csv'):
    """
    Calculate molecular descriptors using padelpy from a list of SMILES IDs.

    Parameters:
    - descriptor_list (list): List of SMILES IDs.
    - output_filename (str, optional): Output filename to save the descriptors (default is 'all_descriptors.csv').

    Returns:
    - None: Saves the calculated descriptors to a CSV file.
    """
    # Create an empty DataFrame to store all the descriptors
    full_df = pd.DataFrame()

    # Loop through each SMILES ID
    for index, smile_id in enumerate(descriptor_list):
        print(f"Calculating descriptors for SMILES ID {index + 1}/{len(descriptor_list)}")
        output_csv = f'{index}.csv'
        from_smiles(smile_id, fingerprints=False, output_csv=output_csv, timeout=600)
        df = pd.read_csv(output_csv)
        full_df = pd.concat([full_df, df], ignore_index=True)

    # Save all descriptors to a single CSV file
    full_df.to_csv(output_filename, index=False)
    print(f"Descriptors saved to {output_filename}")

# Example usage:
if __name__ == "__main__":
    # Replace with your list of SMILES IDs
    descriptor_list = ['SMILES_ID_1', 'SMILES_ID_2', 'SMILES_ID_3']  # Example SMILES IDs
    
    # Call the function to calculate descriptors
    calculate_molecular_descriptors(descriptor_list, output_filename='all_descriptors.csv')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis, boxcox, yeojohnson
from sklearn.preprocessing import StandardScaler
from MultiColumnLabelEncoder import MultiColumnLabelEncoder

def preprocess_data(df):
    """
    Preprocess the dataset for Ligand-based drug designing and QSAR model development.
    
    Steps included:
    1. Handling missing values
    2. Encoding categorical variables
    3. Identifying skewness and kurtosis
    4. Transforming columns to reduce skewness and kurtosis

    Parameters:
    df (DataFrame): Input dataset with [features] Molecular Descriptors of drugs,  and [target variable] Drug dose response data.

    Returns:
    DataFrame: Preprocessed dataset ready for model development.
    """
    
    # QSAR Model Development Stage 1: Handling Missing Values
    print("QSAR MODEL DEVELOPMENT STAGE 1: Handling Missing Values")
    def fill_null_with_mean(df):
        df.fillna(df.mean(), inplace=True)
        df.fillna(0, inplace=True)
    
    fill_null_with_mean(df)
    
    # Alternative approach to fill missing values with SimpleImputer
    from sklearn.impute import SimpleImputer
    print("Using SimpleImputer to fill missing values...")
    imputer = SimpleImputer(strategy='mean')
    df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

    # QSAR Model Development Stage 2: Encoding Categorical Variables
    print("QSAR MODEL DEVELOPMENT STAGE 2: Encoding Categorical Variables")
    mcle = MultiColumnLabelEncoder()
    df = mcle.fit_transform(df)
    
    # QSAR Model Development Stage 3: Identifying Skewness
    print("QSAR MODEL DEVELOPMENT STAGE 3: Identifying Skewness")
    skewness_values = np.apply_along_axis(skew, axis=0, arr=df)
    plt.hist(x=skewness_values)
    plt.title('Skewness Distribution')
    plt.show()

    # QSAR Model Development Stage 4: Identifying Kurtosis
    print("QSAR MODEL DEVELOPMENT STAGE 4: Identifying Kurtosis")
    kurtosis_values = np.apply_along_axis(kurtosis, axis=0, arr=df)
    plt.hist(x=kurtosis_values)
    plt.title('Kurtosis Distribution')
    plt.show()

    # QSAR Model Development Stage 5: Transforming Columns
    print("QSAR MODEL DEVELOPMENT STAGE 5: Transforming Columns to Reduce Skewness and Kurtosis")
    start_column = 12
    X = df.iloc[:, start_column:]

    # Threshold for identifying anomalies in kurtosis
    kurtosis_threshold = 3

    columns_to_boxcox = []
    columns_to_yeojohnson = []
    columns_to_sqrt = []
    columns_to_log = []

    for i, kurt in enumerate(kurtosis_values[start_column:]):
        col_index = start_column + i
        if kurt > kurtosis_threshold:
            try:
                transformed_column, _ = boxcox(X.iloc[:, i] + 1)
                X.iloc[:, i] = transformed_column
                columns_to_boxcox.append(col_index)
            except ValueError:
                transformed_column, _ = yeojohnson(X.iloc[:, i] + 1)
                X.iloc[:, i] = transformed_column
                columns_to_yeojohnson.append(col_index)
        elif kurt < kurtosis_threshold:
            if np.min(X.iloc[:, i]) >= 0:
                transformed_column = np.sqrt(X.iloc[:, i])
                X.iloc[:, i] = transformed_column
                columns_to_sqrt.append(col_index)
            else:
                transformed_column = np.log(X.iloc[:, i] - np.min(X.iloc[:, i]) + 1)
                X.iloc[:, i] = transformed_column
                columns_to_log.append(col_index)

    df.iloc[:, start_column:] = X

    print("Data preprocessing complete. The dataset is ready for QSAR model development.")
    return df

# Example usage
# df = pd.read_csv('your_dataset.csv')
# processed_df = preprocess_data(df)
# print(processed_df.head())


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import silhouette_score

def apply_pca_and_kmeans(df, n_components=2, n_clusters=3):
    """
    Apply PCA for dimensionality reduction and K-Means clustering to analyze cluster effectiveness.

    Parameters:
    df (DataFrame): DataFrame containing the features for clustering.
    n_components (int): Number of components for PCA.
    n_clusters (int): Number of clusters for K-Means.

    Returns:
    DataFrame: DataFrame with cluster labels and silhouette scores.
    """
    # Standardize the data
     print("QSAR MODEL DEVELOPMENT STAGE 6: DIMENSIONALITY REDUCTION")
    scaler = StandardScaler()
    df_std = scaler.fit_transform(df)
    
    # Apply PCA
    pca = PCA(n_components=n_components, random_state=0)
    principal_components = pca.fit_transform(df_std)
    
    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['Cluster_Label'] = kmeans.fit_predict(df_std)
    
    # Calculate silhouette score for each data point
    silhouette_scores = silhouette_score(df_std, df['Cluster_Label'])
    df['Silhouette_Score'] = silhouette_scores
    
    return df, principal_components

def apply_tsne_and_plot(df, n_components=2):
    """
    Apply t-SNE for dimensionality reduction and plot the results.

    Parameters:
    df (DataFrame): DataFrame containing the features for t-SNE.
    n_components (int): Number of components for t-SNE.

    Returns:
    array: Transformed data after t-SNE.
    """
    # Feature selection using SelectKBest and f_regression
    selector = SelectKBest(score_func=f_regression, k=500)  # Adjust 'k' as needed
    X_selected = selector.fit_transform(df, df['continuous_target'])  # Assuming 'continuous_target' is your target variable
    
    # Apply t-SNE
    tsne = TSNE(n_components=n_components, random_state=0)
    X_tsne = tsne.fit_transform(X_selected)
    
    # Plot t-SNE results
    if n_components == 2:
        plt.figure(figsize=(10, 8))
        plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=df['continuous_target'], cmap='viridis')
        plt.colorbar()
        plt.title('t-SNE Plot (2D)')
        plt.xlabel('Component 1')
        plt.ylabel('Component 2')
        plt.show()
    elif n_components == 3:
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], X_tsne[:, 2], c=df['continuous_target'], cmap='magma')
        fig.colorbar(scatter, ax=ax)
        ax.set_title('t-SNE Plot (3D)')
        ax.set_xlabel('Component 1')
        ax.set_ylabel('Component 2')
        ax.set_zlabel('Component 3')
        plt.show()
    
    return X_tsne

# Example usage:
# Assuming 'df' is your DataFrame with features and 'continuous_target' is your continuous target variable

# Apply PCA and K-Means clustering
df_clustered, pca_components = apply_pca_and_kmeans(df.drop(columns=['continuous_target']), n_components=2, n_clusters=3)

# Visualize clusters using PCA
plt.figure(figsize=(10, 8))
for cluster_label in range(3):
    cluster_data = pca_components[df_clustered['Cluster_Label'] == cluster_label]
    plt.scatter(cluster_data[:, 0], cluster_data[:, 1], label=f'Cluster {cluster_label + 1}')

plt.title('Clustering of Drugs Using PCA')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()

# Apply t-SNE and plot
tsne_components = apply_tsne_and_plot(df, n_components=2)

# Save the DataFrame with cluster labels and silhouette scores to an Excel file
df_clustered.to_excel("clustered_drugs_with_scores.xlsx", index=False)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

def perform_grid_search(models_list, x, y, param_grid, cv=5):
    """
    Perform Grid Search CV for multiple models to optimize hyperparameters.
    
    Parameters:
    models_list (dict): Dictionary with model names as keys and model instances as values.
    x (DataFrame): Feature dataset.
    y (Series): Target variable.
    param_grid (dict): Dictionary with model names as keys and parameter grids as values.
    cv (int): Number of cross-validation folds.

    Returns:
    dict: Dictionary with model names as keys and best scores, estimators, and parameters as values.
    """
    results = {}
    
    print("=== QSAR MODEL DEVELOPMENT STAGE 7: Grid Search CV Results ===")
    
    for model_name, model in models_list.items():
        print(f"\nPerforming Grid Search for {model_name}...")
        # Perform GridSearchCV with cross-validation
        grid_search = GridSearchCV(model, param_grid[model_name], cv=cv)
        grid_search.fit(x, y)
        
        # Store the best score, estimator, and parameters
        best_score = grid_search.best_score_
        best_estimator = grid_search.best_estimator_
        best_params = grid_search.best_params_
        
        # Store the results
        results[model_name] = {'Best Score': best_score, 'Best Estimator': best_estimator, 'Best Parameters': best_params}
        
        # Print model evaluation summary
        print(f"Model: {model_name}")
        print(f"Best Score: {best_score:.4f}")
        print(f"Best Estimator: {best_estimator}")
        print(f"Best Parameters: {best_params}")
        print()
    
    return results

def plot_optimization_results(results):
    """
    Plot optimization results for visual analysis.
    
    Parameters:
    results (dict): Dictionary with model names as keys and best scores, estimators, and parameters as values.
    """
    scores = [result['Best Score'] for result in results.values()]
    models = list(results.keys())

    plt.figure(figsize=(10, 6))
    sns.barplot(x=models, y=scores)
    plt.title("Optimization Results for QSAR Models")
    plt.xlabel("Models")
    plt.ylabel("Best Score")
    plt.show()

def validate_model_performance(model, x_train, y_train, x_test, y_test, model_name):
    """
    Validate model performance by plotting regression and residual plots, and performing 8-fold cross-validation.
    
    Parameters:
    model: Trained model to be validated.
    x_train (DataFrame): Training feature dataset.
    y_train (Series): Training target variable.
    x_test (DataFrame): Testing feature dataset.
    y_test (Series): Testing target variable.
    model_name (str): Name of the model for labeling plots and saving files.
    """
    # Predictions
    y_pred = model.predict(x_test)
    
    # Regression Plot
    plt.figure(figsize=(25, 20))
    sns.regplot(x=y_test, y=y_pred, scatter_kws={'s': 15}, ci=95)
    plt.xlabel("Actual Values", fontsize=30)
    plt.ylabel("Predicted Values", fontsize=30)
    plt.title(f"{model_name} Regression Plot - Test Set", fontsize=30)
    plt.grid(True)
    plt.savefig(f"{model_name}_regression_plot.png")
    plt.show()
    
    # Residual Plot
    residuals = y_test - y_pred
    plt.figure(figsize=(30, 20))
    sns.residplot(x=y_pred, y=residuals, scatter_kws={'s': 8, 'color': 'purple'})
    plt.xlabel("Predicted Values", fontsize=24)
    plt.ylabel("Residuals", fontsize=24)
    plt.title(f"{model_name} Residual Plot - Test Set", fontsize=26)
    plt.grid(True)
    plt.savefig(f"{model_name}_residual_plot.png")
    plt.show()
    
    # Cross-Validation
    kf = KFold(n_splits=8, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, x_train, y_train, cv=kf, scoring='neg_mean_squared_error')
    mse_cv = -cv_scores.mean()
    rmse_cv = np.sqrt(mse_cv)
    
    # Regression Metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    evs = explained_variance_score(y_test, y_pred)
    
    print(f"\nValidation Metrics for {model_name}:")
    print("Mean Squared Error (Test Set):", mse)
    print("Root Mean Squared Error (Test Set):", rmse)
    print("Mean Absolute Error (Test Set):", mae)
    print("R-squared (Test Set):", r2)
    print("Explained Variance Score (Test Set):", evs)
    print("Cross-Validated MSE (Train Set):", mse_cv)
    print("Cross-Validated RMSE (Train Set):", rmse_cv)

def build_and_evaluate_models(x_train, y_train, x_test, y_test):
    """
    Build and evaluate models using GridSearchCV for QSAR model development, and validate model performance.

    Parameters:
    x_train (DataFrame): Training feature dataset.
    y_train (Series): Training target variable.
    x_test (DataFrame): Testing feature dataset.
    y_test (Series): Testing target variable.
    """
    # Define models and parameter grid
    models_list = {
        "LinearRegression": LinearRegression(),
        "ElasticNet": ElasticNet(),
        "SGDRegressor": SGDRegressor(),
        "RandomForestRegressor": RandomForestRegressor(random_state=5),
        "DecisionTreeRegressor": DecisionTreeRegressor(random_state=0),
        "GradientBoostingRegressor": GradientBoostingRegressor(),
        "KNeighborsRegressor": KNeighborsRegressor(),
        "SVR": SVR(),
        "MLPRegressor": MLPRegressor(max_iter=500)
    }
    param_grid = {
        "LinearRegression": {},
        "ElasticNet": {'alpha': [0.1, 1], 'l1_ratio': [0.1, 0.5, 0.9]},
        "SGDRegressor": {'alpha': [0.0001, 0.001, 0.01], 'penalty': ['l2', 'l1', 'elasticnet']},
        "RandomForestRegressor": {'n_estimators': [100, 200], 'max_depth': [5, 10], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 5]},
        "DecisionTreeRegressor": {'max_depth': [5, 10], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 5]},
        "GradientBoostingRegressor": {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1], 'max_depth': [3, 5]},
        "KNeighborsRegressor": {'n_neighbors': [3, 5], 'weights': ['uniform', 'distance']},
        "SVR": {'C': [0.1, 1], 'kernel': ['linear', 'rbf']},
        "MLPRegressor": {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam']}
    }

    # Perform GridSearchCV for each model
    results = perform_grid_search(models_list, x_train, y_train, param_grid, cv=5)

    # Plot optimization results
    plot_optimization_results(results)

    # Select the best estimator with the optimum parameters for each model
    best_estimators = {model_name: result['Best Estimator'] for model_name, result in results.items()}

    # Evaluate each model and validate performance
    for name, model in best_estimators.items():
        print(f"\n=== Evaluating {name} ===")
        model.fit(x_train, y_train)
        validate_model_performance(model, x_train, y_train, x_test, y_test, name)
    
    # Identify the best model based on validation metrics
    best_model_name = max(results, key=lambda name: results[name]['Best Score'])
    print(f"\nBest Model Based on Validation Scores: {best_model_name}")

# Example usage with your dataset
# x_train, x_test, y_train, y_test should be defined appropriately
# build_and_evaluate_models(x_train, y_train, x_test, y_test)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def feature_selection_lasso(X, y, alpha=0.01):
     print("QSAR MODEL DEVELOPMENT STAGE 8: FEATURE SELECTION")
    """
    Perform feature selection using Lasso regression.

    Parameters:
    X (DataFrame): DataFrame containing the features.
    y (Series): Series containing the target variable.
    alpha (float): Regularization strength for Lasso.

    Returns:
    DataFrame: DataFrame with selected features based on Lasso coefficients.
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    lasso = Lasso(alpha=alpha, max_iter=10000)
    lasso.fit(X_scaled, y)

    selected_features = X.columns[lasso.coef_ != 0]

    return X[selected_features]

def feature_selection_extra_trees(X, y, n_features_to_select=10):
    """
    Perform feature selection using Extra Trees Regressor.

    Parameters:
    X (DataFrame): DataFrame containing the features.
    y (Series): Series containing the target variable.
    n_features_to_select (int): Number of features to select.

    Returns:
    DataFrame: DataFrame with selected features based on Extra Trees Regressor.
    """
    model = ExtraTreesRegressor(n_estimators=100, random_state=42)

    rfe = RFE(model, n_features_to_select=n_features_to_select)
    rfe.fit(X, y)

    selected_features = X.columns[rfe.support_]

    return X[selected_features]

def feature_selection_rfecv(X, y, cv=5):
    """
    Perform feature selection using Recursive Feature Elimination with Cross-Validation (RFECV).

    Parameters:
    X (DataFrame): DataFrame containing the features.
    y (Series): Series containing the target variable.
    cv (int): Number of cross-validation folds.

    Returns:
    DataFrame: DataFrame with selected features based on RFECV.
    """
    model = ExtraTreesRegressor(n_estimators=100, random_state=42)

    rfecv = RFECV(estimator=model, step=1, cv=cv)
    rfecv.fit(X, y)

    selected_features = X.columns[rfecv.support_]

    return X[selected_features]

# Example usage:
# Assuming 'X' is your DataFrame of features and 'y' is your target variable

# Perform feature selection using Lasso
selected_features_lasso = feature_selection_lasso(X, y)

# Perform feature selection using Extra Trees Regressor
selected_features_extra_trees = feature_selection_extra_trees(X, y)

# Perform feature selection using RFECV
selected_features_rfecv = feature_selection_rfecv(X, y)


In [ ]:
import shap
import matplotlib.pyplot as plt

def shap_analysis(model, X, feature_names, plot_type='summary'):
     print("QSAR MODEL DEVELOPMENT STAGE 9: SHAP (SHAPELY ADDITIVE EXPLAINATIONS) ANALYSIS")
    """
    Perform SHAP analysis to explain feature impact on the model.

    Parameters:
    model: Trained model that supports SHAP values.
    X (DataFrame): DataFrame containing the features.
    feature_names (list): List of feature names.
    plot_type (str): Type of SHAP plot to generate ('summary', 'force', 'bar').

    Returns:
    None
    """
    explainer = shap.Explainer(model)
    shap_values = explainer.shap_values(X)

    if plot_type == 'summary':
        shap.summary_plot(shap_values, X, feature_names=feature_names)
        plt.title('SHAP Summary Plot')
        plt.savefig('shap_summary_plot.png', bbox_inches='tight')
        plt.show()
    elif plot_type == 'force':
        shap.initjs()
        shap.force_plot(explainer.expected_value, shap_values, X)
    elif plot_type == 'bar':
        shap.summary_plot(shap_values, X, plot_type='bar', feature_names=feature_names)
        plt.title('SHAP Feature Importance')
        plt.savefig('shap_feature_importance.png', bbox_inches='tight')
        plt.show()

# Example usage:
# Assuming 'model' is your trained model, 'X' is your DataFrame of features, and 'feature_names' is a list of feature names

# Perform SHAP analysis with summary plot
shap_analysis(model, X, feature_names=list(X.columns), plot_type='summary')
